In [1]:
import torch
import torch.nn as nn
import numpy as np
import math

In [2]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.0, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x:torch.nn) -> torch.nn:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x.transpose(0,1)
        x = x + self.pe[:x.size(0)]
        return self.dropout(x).transpose(0,1)
    
class AttentionBlock(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, num_heads=8, dropout=0, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        self.feed_forward = nn.Sequential(
            nn.Linear(d_model , d_model * 2),
            nn.GELU(),
            nn.Linear(d_model * 2, d_model),
        )

    def forward(self , target , memory , **kwargs):
        target = self.norm1(target + self.self_attn(target , memory , memory , need_weights=False)[0])
        target = self.norm2(target + self.feed_forward(target))
        return target

class EmbeddingModel(nn.Module):
    def __init__(self,):
        super().__init__()
        self.hidden_size = 4
        self.atten_size = 384

        self.input_layer = nn.Sequential(
            nn.Linear(4 , self.atten_size),
            nn.GELU(),
        )

        self.position_encoding = PositionalEncoding(self.atten_size , dropout=0 , max_len=361)

        encoder_layer = nn.TransformerEncoderLayer(d_model=self.atten_size,  nhead=6 , dim_feedforward=1024 , batch_first=True , activation="gelu")
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=5)

    def forward(self , x):
        # [batch , w , h , dim]
        x = self.input_layer(x).flatten(1,2)

        # [batch , w * h , dim]
        x = self.position_encoding(x)
        x = self.transformer_encoder(x)
        return x

class PlayStyleModel(nn.Module):
    def __init__(self,):
        super().__init__()
        self.hidden_size = 32
        self.atten_size = 384

        self.embedding_model = EmbeddingModel()
        
        self.atten_layer = nn.TransformerDecoder(AttentionBlock(self.atten_size), num_layers=3)
        self.query = nn.Parameter(torch.rand(1,self.atten_size,))

        self.output_layer = nn.Sequential(
            nn.Linear(self.atten_size , 3),
            nn.Softmax(dim=1),
        )

    def forward(self , x):
        x = self.embedding_model(x)

        query = self.query.repeat(x.shape[0] , 1 , 1)

        x = self.atten_layer(query , x).squeeze(1)
        return self.output_layer(x)

class MoveModel(nn.Module):
    def __init__(self,):
        super().__init__()
        self.atten_size = 512

        self.input_layer = nn.Sequential(
            nn.Linear(4 , self.atten_size),
            nn.GELU(),
        )

        self.position_encoding = PositionalEncoding(self.atten_size , dropout=0 , max_len=361)

        encoder_layer = nn.TransformerEncoderLayer(d_model=self.atten_size,  nhead=8 , dim_feedforward=2048 , batch_first=True , activation="gelu")
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=5)

        self.output_linear = nn.Linear(self.atten_size , 1)

    def forward(self , x):
        # [batch , w , h , dim]
        x = self.input_layer(x).flatten(1,2)

        # [batch , w * h , dim]
        x = self.position_encoding(x)
        x = self.transformer_encoder(x)

        return self.output_linear(x).squeeze(-1)
    

# Create a Submission File 

The submitted file must include all the predictions from Dan, Kyu, and Play Style testing CSVs. Therefore, you must train your models before submitting any file. If your model is not trained for Dan, Kyu, or Play Style, check out the previous tutorials about how to train your model or just simply use the baseline models (model_dan_tutorial.h5, model_kyu_tutorial.h5, model_playstyle.h5) 

**PublicUpload.csv** must be in the following form:
```
DTPU0000000001,id,qr,pq,pd,ab
DTPU0000000002,ao,ab,ha,ff,qd
DTPU0000000003,qd,gd,fh,ed,fa
DTPU0000000004,pr,ba,dq,hh,aj
DTPU0000000005,ph,jh,af,df,gj
.
.
.
KTPU0000000001,id,qr,pq,pd,ab
KTPU0000000002,ao,ab,ha,ff,qd
KTPU0000000003,qd,gd,fh,ed,fa
KTPU0000000004,pr,ba,dq,hh,aj
KTPU0000000005,ph,jh,af,df,gj
.
.
.
PSTPU0000000001_79,2
PSTPU0000000001_55,3
PSTPU0000000001_105,2
PSTPU0000000002_37,1
PSTPU0000000002_113,3
```

**For Dan and Kyu:**
- Column 1: Game ID
- Column 2: Predicted Moves, up to 5 predictions for each game

**For PlayStyle**:
- Column 1: Game ID
- Column 2: Predicted Game Style

The code block below is copied from the tutorial notebooks and it includes the pre-processing methods of the trained models. You might need to change the functions below with respect to your models.

In [3]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}

def prepare_input_for_dan_kyu_models(moves):
    x = torch.zeros((19,19,4) , dtype=torch.short)
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = torch.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_input_for_playstyle_model(moves):
    moves = list(filter(None, moves))
    x = torch.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = torch.where(x[:,:,2] == 0, 1, 0)
    return x

def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

## Save Predictions for Dan
The code block below is to use **dan model** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [4]:
from tqdm import tqdm
# Load your own model. Here we use the baseline model
model = MoveModel()
model.load_state_dict(torch.load("model/play_dan_xxlarge_backup2.pt") , strict=True)

# Load the corresponding dataset
df = open('./CSVs/29_Private Testing Dataset_v2/dan_test_private.csv').read().splitlines()
df.extend(open('./CSVs/dan_test_public.csv').read().splitlines())
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list))

x_testing = torch.stack(x_testing)

model.eval()
model.cuda()
with torch.no_grad():
    dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_testing) , batch_size=256 , shuffle=False)
    with torch.cuda.amp.autocast(dtype=torch.float16 , enabled=False):
        predictions = torch.cat([model(x.float().cuda()) for (x,) in tqdm(dataloader)] , dim=0).cpu().numpy()
    # predictions = model.predict(x_testing)
    prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_private_submission.csv','w') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

100%|██████████| 86/86 [00:54<00:00,  1.58it/s]


## Save Predictions for Kyu
The same code block applies to Kyu by changing the model and dataset

In [5]:
# Load your own model. Here we use the baseline model
model = MoveModel()
model.load_state_dict(torch.load("model/play_kyu_xxlarge_backup.pt") , strict=True)

# Load the corresponding dataset
df = open('./CSVs/29_Private Testing Dataset_v2/kyu_test_private.csv').read().splitlines()
df.extend(open('./CSVs/kyu_test_public.csv').read().splitlines())
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list))

x_testing = torch.stack(x_testing)

model.eval()
model.cuda()
with torch.no_grad():
    dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_testing) , batch_size=256 , shuffle=False)
    with torch.cuda.amp.autocast(dtype=torch.float16 , enabled=False):
        predictions = torch.cat([model(x.float().cuda()) for (x,) in tqdm(dataloader)] , dim=0).cpu().numpy()
    # predictions = model.predict(x_testing)
    prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_private_submission.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

100%|██████████| 89/89 [00:55<00:00,  1.61it/s]


## Save Predictions for PlayStyle
Playstyle requires only one predictions so the code here is a simple version of above

In [6]:
# Load your own model. Here we use the baseline model
model = PlayStyleModel()
model.load_state_dict(torch.load("model/play_style.pt") , strict=True)
model.eval()
model.cuda()

# Load the corresponding dataset
df = open('./CSVs/29_Private Testing Dataset_v2/play_style_test_private.csv').read().splitlines()
df.extend(open('./CSVs/play_style_test_public.csv').read().splitlines())
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []
for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_playstyle_model(moves_list))

x_testing = torch.stack(x_testing)
with torch.no_grad():
    dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_testing) , batch_size=256 , shuffle=False)
    with torch.cuda.amp.autocast(dtype=torch.float16 , enabled=False):
        predictions = torch.cat([model(x.float().cuda()) for (x,) in tqdm(dataloader)] , dim=0).cpu().numpy()
    prediction_numbers = np.argmax(predictions, axis=1)

with open('./public_private_submission.csv','a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)

100%|██████████| 79/79 [00:30<00:00,  2.60it/s]


Now you can upload your file from the competition website! (https://tbrain.trendmicro.com.tw/Competitions/Details/29)